<a href="https://colab.research.google.com/github/zmohaghegh/Zahra-PDF-Chat-RAG/blob/main/Zahra_RAG_Chatbot_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -q langchain langchain-community langchain-text-splitters pypdf sentence-transformers faiss-cpu transformers gradio
import torch
import gradio as gr
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from transformers import pipeline

# --- تنظیمات اولیه ---
# تشخیص سخت‌افزار (GPU یا CPU)
device_id = 0 if torch.cuda.is_available() else -1
device_name = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Running on: {device_name}")

# بارگذاری مدل Embedding (رایگان و سبک)
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",
    model_kwargs={'device': device_name}
)

# بارگذاری مدل پاسخ‌دهنده (Flan-T5)
print("Loading LLM model...")
qa_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    device=device_id
)

# --- توابع اصلی سیستم ---

def process_pdf_and_chat(file_obj, user_query):
    """
    این تابع فایل PDF را دریافت کرده، آن را ایندکس می‌کند و سپس به سوال کاربر پاسخ می‌دهد.
    """
    if file_obj is None:
        return "زهرا جان، لطفاً ابتدا یک فایل PDF آپلود کن."
    if not user_query:
        return "لطفاً سوال خود را بپرسید."

    try:
        # ۱. بارگذاری محتوای PDF
        loader = PyPDFLoader(file_obj.name)
        pages = loader.load()

        # ۲. قطعه‌بندی متن (Chunking) برای پردازش بهتر
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        docs = text_splitter.split_documents(pages)

        # ۳. ذخیره‌سازی در پایگاه داده برداری (Vector Database)
        vectorstore = FAISS.from_documents(docs, embeddings)

        # ۴. بازیابی بخش‌های مرتبط (Retrieval)
        relevant_docs = vectorstore.similarity_search(user_query, k=3)
        context = " ".join([d.page_content for d in relevant_docs])

        # ۵. تولید پاسخ نهایی (Generation)
        prompt = f"Answer the question using the context below.\nContext: {context}\nQuestion: {user_query}"
        result = qa_pipeline(prompt, max_length=150)

        return result[0]['generated_text']

    except Exception as e:
        return f"خطایی رخ داد: {str(e)}"

# --- طراحی رابط کاربری (Gradio UI) ---

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(f"""
    # 🤖 سیستم چت هوشمند زهرا (Zahra's RAG System)
    ### توسعه داده شده با LangChain، FAISS و Transformers
    این پروژه یک سیستم **Retrieval-Augmented Generation** است که به شما اجازه می‌دهد با هر فایل PDF چت کنید.
    """)

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(label="آپلود فایل PDF", file_types=[".pdf"])
        with gr.Column(scale=2):
            query_input = gr.Textbox(label="سوال خود را از محتوای فایل بپرسید:", placeholder="مثلاً: What is the summary of this document?")
            output_text = gr.Textbox(label="پاسخ هوش مصنوعی:", interactive=False)

    submit_btn = gr.Button("تحلیل و پاسخدهی", variant="primary")

    # تعریف اکشن دکمه
    submit_btn.click(
        fn=process_pdf_and_chat,
        inputs=[file_input, query_input],
        outputs=output_text
    )

    gr.Markdown("--- \n *ساخته شده توسط زهرا برای نمایش قدرت RAG در پردازش اسناد.*")

# اجرا (اگر share=True باشد، لینک عمومی می‌سازد)
if __name__ == "__main__":
    demo.launch(share=True)

Running on: cpu
Loading LLM model...


Device set to use cpu
/tmp/ipython-input-1652981425.py:69: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f9523693fa6a2e5a7b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
